In [63]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array


In [64]:
pre_image_path='C:\\Users\\akasw\\Desktop\\Small_Scale_Dataset\\test\\A'
post_image_path='C:\\Users\\akasw\\Desktop\\Small_Scale_Dataset\\test\\B'
label_dir_path='C:\\Users\\akasw\\Desktop\\Small_Scale_Dataset\\test\\label'

In [65]:
pre_image=[]
post_image=[]
label_image=[]
for i in sorted(os.listdir(pre_image_path)):
  image_path=os.path.join(pre_image_path,i)
  img=cv2.imread(image_path)
  pre_image.append(img)

for i in sorted(os.listdir(post_image_path)):
  image_path=os.path.join(test_post_image_path,i)
  img=cv2.imread(image_path)
  post_image.append(img)

for i in sorted(os.listdir(label_dir_path)):
  image_path=os.path.join(label_dir_path,i)
  img=cv2.imread(image_path)
  label_image.append(img)

label=np.array(label_image)


In [66]:
concatenated_images = np.concatenate([pre_image, post_image], axis=3)
concatenated_images.shape

(50, 1024, 1024, 6)

In [67]:
concatenated_images=concatenated_images/255.0
label=label/255.0


In [69]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(concatenated_images, label, test_size=0.33, random_state=0)

In [70]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(1022, 1022, 3))

# Freeze the base model's layers to prevent them from being updated during fine-tuning
for layer in base_model.layers:
    layer.trainable = False

# Create the input layer for the model
input_layer = tf.keras.layers.Input(shape=(1024, 1024, 6))

# Apply a convolutional layer to transform the input from 6 channels to 3 channels
x = tf.keras.layers.Conv2D(3, 3, activation='relu')(input_layer)

# Pass the transformed input through the base model
x = base_model(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
flatten = Flatten()(x)
predictions = Dense(1024 * 1024 * 3, activation='sigmoid')(flatten)

# Create the fine-tuned model
model = Model(inputs=input_layer, outputs=predictions)

# Reshape the target labels to match the expected shape
Y_train_reshaped = np.reshape(Y_train, (Y_train.shape[0], 1024 * 1024 * 3))
Y_test_reshaped = np.reshape(Y_test, (Y_test.shape[0], 1024 * 1024 * 3))

# Compile the model
optimizer = Adam(learning_rate=0.001)
loss = BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Reduce the batch size
batch_size = 11

# Train the model with the reshaped target labels and reduced batch size
model.fit(X_train, Y_train_reshaped, epochs=10, batch_size=batch_size, validation_data=(X_test, Y_test_reshaped))

In [ ]:
# Make predictions on test data
predictions = model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt

# Get training history
history = model.fit(X_train, Y_train_reshaped, epochs=10, batch_size=batch_size, validation_data=(X_test, Y_test_reshaped))

# Plot accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
